## Assignment 1 
#### CS395: Deep learning
#### 2/26/18
#### By Alex Ram

### The code bellow is used for all three parts

In [ ]:
#imports
from keras.applications.resnet50 import ResNet50 
from keras.preprocessing.image import img_to_array
from IPython.display import Image
import cv2
import random
import numpy as np
from keras.applications.resnet50 import preprocess_input
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D

In [ ]:
#code bellow was inspired by this tutorial: https://www.pyimagesearch.com/2017/12/11/image-classification-with-keras-and-deep-learning/
#This will take the images and covert it to an array, also the labels
#path to the pictures
PATH = "../../png/"
trainData = []
trainLabels = []

testData = []
testLabels = []
 
# grab the image paths and randomly shuffle them
imagePaths = open(PATH + "filelist.txt").readlines()

def convertImageToArray(imagePath):
    image = cv2.imread(PATH + imagePath.strip())
    image = cv2.resize(image, (224, 224))
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)
    image = preprocess_input(image)
    return image

#len(imagePaths)
for i in range(160):
    if i % 80 < 60:
        #image coverted to array and appended to data set
        trainData.append(convertImageToArray(imagePaths[i]))
        
        #labels of images appended to labels dataset
        trainLabels.append(imagePaths[i].split("/"))
        trainLabels[-1][1] = trainLabels[-1][1].strip()
    else:
        testData.append(convertImageToArray(imagePaths[i]))
        
        testLabels.append(imagePaths[i].split("/"))
        testLabels[-1][1] = testLabels[-1][1].strip()
        
#covert label lists to numpy arrays to use later
trainLabels = np.array(trainLabels)
testLabels = np.array(testLabels)
#get a set of just each type of lable
uniqueLabels = set(trainLabels[:,0])

## Part 1:
### Using a pretrained network on imageNet to classify the sketches data set by training a classifier 

In [ ]:
#model found here: https://keras.io/applications/#resnet50
model = ResNet50(include_top=False, weights='imagenet')

In [ ]:
trainFeatures = []
testFeatures = []

#feature extraction from the image arrays using the resnet50 model
for i in range(len(trainData)):
    trainFeatures.append(model.predict(trainData[i]))
    
for i in range(len(testData)):
    testFeatures.append(model.predict(testData[i]))

#squeeze 'em
trainFeatures = np.squeeze(trainFeatures)
testFeatures = np.squeeze(testFeatures)

In [ ]:
#code bellow inspired by https://machinelearningmastery.com/multi-class-classification-tutorial-keras-deep-learning-library/

#Creates the classifier
def createClassifier():
    model = Sequential()
    model.add(Dense(2, input_dim=2048, activation='softmax'))
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# encode class values as integers
def convertToDummy(labels):
    encoder = LabelEncoder()
    encoder.fit(labels[:,0])
    encoded_Y = encoder.transform(labels[:,0])
    # convert integers to dummy variables (i.e. one hot encoded)
    dummy_y = np_utils.to_categorical(encoded_Y)
    return dummy_y

trainDummyLabels = convertToDummy(trainLabels)
testDummyLabels = convertToDummy(testLabels)

estimator = createClassifier()
estimator.fit(trainFeatures, trainDummyLabels, validation_data=(testFeatures, testDummyLabels), epochs=50, batch_size=10)


## Results

## Part 2:
### Allowing the entire pretrained network to be trained on the sketches data set

In [ ]:
#I'm taking the resnet50 model defined in the previous part and adjustiong to have the classifier added to it.
#This will allow the entire network to be trained in one go
#code inspired by https://keras.io/applications/#resnet50

x = model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(2, activation='softmax')(x)

# this is the model we will train
modelWithClassifier = Model(inputs=model.input, outputs=predictions)

In [ ]:
modelWithClassifier.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

#squeeze/ convert to numpy array so the network can take in the data
trainData = np.squeeze(np.array(trainData))
testData = np.squeeze(np.array(testData))

#train the netwrok
modelWithClassifier.fit(trainData, trainDummyLabels, validation_data=(testData, testDummyLabels), epochs=50, batch_size=50)

## Results:

## Part 3:
### Initializing the network with _____ and training from scratch on the sketches data set

In [ ]:
modelFromScratch = ResNet50(include_top=True, weights=None, classes=2)
modelFromScratch.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
modelFromScratch.fit(trainData, trainDummyLabels, validation_data=(testData, testDummyLabels), epochs=50, batch_size=50)

## Results:
